In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map 

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [5]:
print(ee.batch.Task.list())

[<Task EXPORT_IMAGE: post_water9 (COMPLETED)>, <Task EXPORT_IMAGE: post_water (COMPLETED)>, <Task INGEST_TABLE: Ingest table: "projects/earthengine-legacy/assets/users/311605001111/province" (COMPLETED)>, <Task INGEST_TABLE: Ingest table: "projects/earthengine-legacy/assets/users/311605001111/samples_land" (COMPLETED)>]


In [10]:
Camana_valley = ee.Geometry.Rectangle([-72.778645, -16.621663, -72.66865, -16.57553])
center = Camana_valley.centroid().getInfo()['coordinates']
center.reverse()
print(a)
print(center)

None
[-16.598602803414238, -72.72364749999926]


# ee.Reducer.mode()的用法

In [19]:
lt = ee.List([0,0,0,0,0,1,1,1,1,2,2])
mode = ee.Reducer.mode(2,1)
lt_mode = lt.reduce(mode)
print(lt_mode.getInfo())

0


# 查看图像的投影和像元大小

In [ ]:
# projection and 
image = ee.Image('MODIS/MOD13A1/MOD13A1_005_2014_05_09')
visParams = {'min': 0.15, 'max': 0.7} 
Map.setCenter(-77.5127,43.2642,11) 
Map.addLayer(image, visParams, 'original')

print(image.bandNames().getInfo())
print('Projection and transformation information:', image.projection().getInfo()['crs'])
print('Pixel size in meters:', image.projection().nominalScale().getInfo())

reprojected = image.reproject('EPSG:4326')
Map.addLayer(reprojected, visParams, 'Reprojected')

# 图像校正

In [ ]:
# registerion
image1 = ee.Image('SKYSAT/GEN-A/PUBLIC/ORTHO/MULTISPECTRAL/s01_20150502T082736Z')
image2 = ee.Image('SKYSAT/GEN-A/PUBLIC/ORTHO/MULTISPECTRAL/s01_20150305T081019Z')

image1Orig = image1.resample('bicubic')
image2Orig = image2.resample('bicubic')
 
image1RedBand = image1Orig.select('R')
image2RedBand = image2Orig.select('R')
 
displacement = image2RedBand.displacement(**{
    'referenceImage': image1RedBand,  
    'maxOffset': 50.0,      
    'patchWidth': 100.0   
})  
registered = image2Orig.displace(displacement)
 
visParam = {'bands': ['R', 'G', 'B'], 'max': 4000}
Map.addLayer(image1Orig, visParam, 'Reference'); 
Map.addLayer(image2Orig, visParam, 'Before Registration') 
Map.addLayer(registered, visParam, 'After Registration')

# 计算熵

In [ ]:
image = ee.Image('USDA/NAIP/DOQQ/m_4307663_se_18_1_20130619')

Map.setCenter(-76.13, 43.03,16)
Map.addLayer(image, {'max': 255}, 'Input image')

nir = image.select('N')
square = ee.Kernel.square(**{'radius': 3})
entropy = nir.entropy(square)

Map.addLayer(entropy,{'min': 1, 'max': 5, 'palette': ['0000CC', 'CC0000']}, 'entropy')

# 直方图
histogram = entropy.select('N').reduceRegion(**{
    'reducer': ee.Reducer.histogram(100),
    'geometry': image.geometry(), 
    'scale': 100,
    'bestEffort': True
})
print(histogram.getInfo())

# 图像融合

In [ ]:
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_121038_20160115')
Map.addLayer(image,{'bands': ['B4', 'B3', 'B2'],'min':0,'max':3000}, 'RGB image')
hsv = image.select(['B4', 'B3', 'B2']).rgbToHsv()

combined = ee.Image.cat([hsv.select('hue'), hsv.select('saturation'), image_toa.select('B8')])

sharpened = ee.Image(combined).hsvToRgb()
Map.centerObject(image,8)
Map.addLayer(sharpened,'','pan-sharpened')

# 边缘检测

In [ ]:
image = ee.Image('LANDSAT/LC08/C01/T1/LC08_015030_20140307').select("B8")
Map.setCenter(-76.1467, 43.0458,12)
Map.addLayer(image, '', 'input image')

canny = ee.Algorithms.CannyEdgeDetector(**{'image': image, 'threshold': 200, 'sigma': 1 })
Map.addLayer(canny, {}, 'canny')

hough = ee.Algorithms.HoughTransform(canny, 256, 7000, 80)
Map.addLayer(hough, {}, 'hough')

# 定义无geometry的要素

In [ ]:
ab = ee.Feature(None,{'time': 0,'area': 1,'area_smoothed': 2})
print(ab.getInfo())

# 提取字典的属性

In [ ]:
print(ee.Date('2020-01-01').getInfo())
print(ee.Date('2020-01-01').getInfo()['value'])

# 定义一个要素，没有几何属性

In [ ]:
a = ee.Feature(None, {'foo': 1})
print(a.getInfo())

# 移除要素的某个属性

In [ ]:
# feat为输入要输
def removeProperty(feat):
    selectProperties = ee.Feature(feat).propertyNames().filter(ee.Filter.neq('item', 'waterclass'))
    return feat.select(selectProperties)

# 区域最大值最小值

In [ ]:
minmax = allPixel.reduceRegion(**{
    'reducer': ee.Reducer.minMax(),
    'geometry': roi,
    'scale': 30,
    'maxPixels': 1e14,
})
print(minmax.getInfo())

# 计算面积

In [ ]:
image_area = image.select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
imageareas = image_area.reduceRegion(**{
    'reducer': ee.Reducer.sum(),
    'geometry': image.geometry(),
    'scale': 100,
    'maxPixels': 1e14,
}) 

# 获取无效像元所在区域

In [ ]:
def maskL8sr_reverse(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask.remap([0,1],[1,0]))

# 加载sentinel-2

In [ ]:
# sentinel-2单景影像的加载，system:index
s2 = ee.Image('COPERNICUS/S2_SR/20200805T030549_20200805T030831_T49RGP')
Map.addLayer(s2, {'bands': ['B8', 'B11', 'B4'], 'max': 4000}, 'S2')
Map.centerObject(s2,8)

# 加载矢量数据

# 下载影像(本地)

In [ ]:
geemap.ee_export_image(ee_object=awei_water,filename='awei_water.tif',scale=30,region=roi.geometry(),file_per_band=False,)

In [ ]:
# shp格式
selector = ['Image_id','B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI','waterclass']
geemap.ee_export_vector_to_drive(points_collection, description=label, folder=folder, file_format='shp', selectors=selector)

# 下载影像（asset）

In [ ]:
task = ee.batch.Export.image.toAsset(**{
    'image': validPixel,
    'description': 'validPixel',
    'assetId': 'users/311605001111/validPixel_wuhan',
    'scale': 30,
    'region': roi,
    'maxPixels':1e13,
})
task.start()

In [ ]:
task = ee.batch.Export.table.toAsset(**{
    'collection': points_collection,
    'description':'daqing2015',
    'assetId': 'users/311605001111/daqing_2015'
})
task.start()